In [1]:
from gurobipy import Model, GRB, quicksum

In [2]:
# Data from the problem
probabilities = [0.09, 0.12, 0.10, 0.05, 0.16, 0.14, 0.03, 0.08, 0.05, 0.05, 0.04, 0.03, 0.02, 0.01, 0.02, 0.01]
demands = [90, 95, 100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165]
supplier_costs = [120, 105, 110]  # Costs for Phil & Sebastian, Rosso, and Monogram respectively
advance_order_cost = 95
num_scenarios = len(probabilities)
min_order_quantities = [0, 70, 40]  # Minimum order quantities for Phil & Sebastian, Rosso, and Monogram

In [3]:
# Create a new model
m = Model('CoffeeSupplyOptimization')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-15


In [4]:
# Decision variables
x = m.addVar(vtype=GRB.CONTINUOUS, name="x")  # Gallons of coffee ordered in advance
# Gallons of coffee ordered from local suppliers in each scenario
y = m.addVars(3, num_scenarios, vtype=GRB.CONTINUOUS, name="y")

In [5]:
# Objective Function: Minimize the expected cost of ordering coffee
m.setObjective(advance_order_cost * x + quicksum(probabilities[n] * quicksum(supplier_costs[i] * y[i,n] for i in range(3)) for n in range(num_scenarios)), GRB.MINIMIZE)

In [6]:
# Constraints
# Ensure demand is met for each scenario
for n in range(num_scenarios):
    m.addConstr(x + quicksum(y[i,n] for i in range(3)) >= demands[n], "Demand_Scenario_%s" % n)

# Supplier minimum order constraints for Rosso and Monogram
for i in range(1, 3):  # Skip Phil & Sebastian since it has no minimum
    for n in range(num_scenarios):
        m.addGenConstrIndicator(y[i,n], True, y[i,n] >= min_order_quantities[i], name=f"Min_Order_{i}_{n}")


In [7]:
# Optimize the model
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 16 rows, 49 columns and 64 nonzeros
Model fingerprint: 0x6a6e1f80
Model has 32 general constraints
Variable types: 49 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+01, 2e+02]
  GenCon rhs range [4e+01, 7e+01]
  GenCon coe range [1e+00, 1e+00]
Found heuristic solution: objective 8.600000e+11
Presolve removed 16 rows and 49 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 11437 8.6e+11 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.143700000000e+04, best bound 1.143700000000e+04, 

In [8]:
# Output solution
if m.status == GRB.OPTIMAL:
    print(f"Optimal amount of coffee to order in advance: {x.X} gallons.")
    for i in range(3):
        for n in range(num_scenarios):
            if y[i,n].X > 0:
                print(f"Order {y[i,n].X} gallons from supplier {i+1} in scenario {n+1}.")
else:
    print("No optimal solution found.")

Optimal amount of coffee to order in advance: 95.0 gallons.
Order 5.0 gallons from supplier 1 in scenario 3.
Order 10.0 gallons from supplier 1 in scenario 4.
Order 15.0 gallons from supplier 1 in scenario 5.
Order 20.0 gallons from supplier 1 in scenario 6.
Order 25.0 gallons from supplier 1 in scenario 7.
Order 30.0 gallons from supplier 1 in scenario 8.
Order 35.0 gallons from supplier 1 in scenario 9.
Order 40.0 gallons from supplier 1 in scenario 10.
Order 45.0 gallons from supplier 1 in scenario 11.
Order 50.0 gallons from supplier 1 in scenario 12.
Order 55.0 gallons from supplier 1 in scenario 13.
Order 60.0 gallons from supplier 1 in scenario 14.
Order 65.0 gallons from supplier 1 in scenario 15.
Order 70.0 gallons from supplier 1 in scenario 16.
